In [177]:
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import metadata_parser
import requests
import logging
logger = logging.getLogger('django')
# logger.info('file: %s' % file)

# meta site
# url = 'http://www.naver.com'
# url = 'http://poiemaweb.com/angular-component-style'
# url = 'http://terms.naver.com/'
# url = 'https://twitter.com/sheispuzzled/status/973207418548072448'
# url = 'http://akul.me/blog/2016/beautifulsoup-cheatsheet/'

# naver blog
# url = 'https://blog.naver.com/bessgo/221227429246'
# url = 'https://blog.naver.com/juneeeeeee/221225818069'

# daum blog
# url = 'http://blog.daum.net/leehungkyu/2176?bt_nil_d=0312_4'

#egloos
# url = 'http://runtorun.egloos.com/1263788'




def summary(url):
    parsed_url = urlparse(url)     

    naver_attrs = {'id':'mainFrame'}
    daum_attrs = {'name':'BlogMain'}

    except_url ={
        # naver
        'blog.naver.com':naver_attrs,
        'm.blog.naver.com':naver_attrs,
        # daum
        'blog.daum.net': daum_attrs,
        'm.blog.daum.net': daum_attrs,
    }

    def get_html_and_bs(url):   
        headers = {'Referer': url,'user-agent': 'my-app/0.0.1'}
        html = requests.get(url, headers=headers).text
        bs = BeautifulSoup(html, 'html.parser')
        return {'html':html, 'bs': bs}

    # ParseResult(scheme='http', netloc='www.naver.com', path='', params='', query='', fragment='')
    def is_except_url(parsed_url):   
        if parsed_url.netloc in except_url.keys():        
            return except_url[parsed_url.netloc]           
        return False

    def get_except_url_in_source(url, parsed_url):
        attrs = is_except_url(parsed_url)
        bs =  get_html_and_bs(url)['bs']
    #     set_html_and_bs(url)
        source_url = bs.find("frame", attrs )['src']
        source_url = parsed_url.scheme + '://' + parsed_url.netloc + source_url
        return source_url

    def status_check(url):
        r = requests.head(url)
        if r.status_code == requests.codes.ok:
            return True
        return False


    def get_metadata_parser(url):
        page = metadata_parser.MetadataParser(url)
        data = {
                'title': page.get_metadata('title'),
                'url': page.get_metadata('url'),
                'image': page.get_metadata('image'),
                'description': page.get_metadata('description'),
            }
        return data


    def custom_parser(bs = get_html_and_bs(url)['bs']):
        title = bs.find('title').text
        body = bs.find_all('p', limit=10)
        strip_body = ''
        for b in body:
            strip_body += b.text
        return {'title': title, 'body': strip_body}

    if is_except_url(parsed_url):
        data = get_metadata_parser(get_except_url_in_source(url, parsed_url))
    else:        
        data = get_metadata_parser(url)



    if not data['title']:
        data['title'] = custom_parser()['title']

    if not data['description']:
        data['description'] = custom_parser()['body']

    logger.info('data: %s' % data)
    
    meta_data = {
        'url': data['url'],
        'domain': parsed_url.netloc,
        'title': data['title'],
        'description': data['description'],
        'image': data['image'],
#         'source_url':,
    }
    
    
    return meta_data

summary(url)

C:\Users\whattheshim\Envs\bookmark\lib\site-packages\metadata_parser\__init__.py:1005: FutureWarning: `search_head_only` was not provided and defaulting to `True` Future versions will default to `False`.
  warn_future("""`search_head_only` was not provided and defaulting to `True` """
data: {'title': 'Angular Component - Style | PoiemaWeb', 'url': 'http://poiemaweb.com/angular-component-style', 'image': 'http://poiemaweb.com/img/poiemaweb.jpg', 'description': 'Angular 컴포넌트는 동작 가능한 하나의 부품으로 다른 컴포넌트에 간섭을 받지 않는 독립된 스코프의 스타일 정보를 갖는다. 다시 말해 컴포넌트에서 정의한 스타일은 그 컴포넌트에서만 유효하다. 스타일을 정의하는 방법은 @Component 데코레이터의 메타데이터 객체의 styles 프로퍼티에 직접 정의하는 방법과 styleUrls 프로퍼티에 외부 CSS 파일의 경로를 정의하는 방법이 있다.'}


{'description': 'Angular 컴포넌트는 동작 가능한 하나의 부품으로 다른 컴포넌트에 간섭을 받지 않는 독립된 스코프의 스타일 정보를 갖는다. 다시 말해 컴포넌트에서 정의한 스타일은 그 컴포넌트에서만 유효하다. 스타일을 정의하는 방법은 @Component 데코레이터의 메타데이터 객체의 styles 프로퍼티에 직접 정의하는 방법과 styleUrls 프로퍼티에 외부 CSS 파일의 경로를 정의하는 방법이 있다.',
 'domain': 'poiemaweb.com',
 'image': 'http://poiemaweb.com/img/poiemaweb.jpg',
 'title': 'Angular Component - Style | PoiemaWeb',
 'url': 'http://poiemaweb.com/angular-component-style'}